In [35]:
# from keras.models import Sequential
# from keras.engine.training import Model
# from tensorflow.keras.layers import *
# from keras.regularizers import l2
# from keras.losses import binary_crossentropy
# from keras.applications.vgg16 import VGG16
# from keras.applications.resnet50 import ResNet50
# from keras.applications import xception, inception_v3
# from keras.applications import inception_v3
from tensorflow import keras
from keras.layers import *

import numpy as np

import os, gc

In [36]:
vocab_size = 2500

In [37]:
bow_inputs = Input(shape=(vocab_size,))
# variety_inputs = layers.Input(shape=(num_classes,))
# merged_layer = layers.concatenate([bow_inputs, variety_inputs])
dense_layer = Dense(256, activation='relu')(bow_inputs)
predictions = Dense(1)(dense_layer)
wide_model = keras.Model(inputs=bow_inputs, outputs=predictions)
print(wide_model.output.shape)

(?, 1)


In [38]:
wide_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [39]:
vgg16_notop = VGG16(weights='/Users/Account/Drink/kaggle/avito/pretrained_models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', 
              include_top=False, 
              input_shape=(224, 224, 3))
for layer in vgg16_notop.layers[:-5]:
    layer.trainable = False
vgg16_out = Flatten()(vgg16_notop.output)
vgg16_out = Dense(1, activation = 'relu')(vgg16_out)
vgg16 = Model(inputs = vgg16_notop.input, outputs = vgg16_out)
print(vgg16_out.shape)

(?, 1)


In [40]:
vgg16.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [41]:
merged_out = layers.concatenate([wide_model.output, vgg16.output])
merged_out = layers.Dense(1)(merged_out)
combined_model = Model(wide_model.input + [vgg16.input], merged_out)
combined_model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])

AttributeError: 'Dense' object has no attribute 'outbound_nodes'

In [16]:
def make_final_model(pretrained_model, vec_size, do=0.5, l2_strength=1e-5):
    """
    pretrained_model : The VGG conv only part with loaded weights
    vec_size : The size of the vectorized text vector coming from the bag of words 
    n_classes : How many classes are you trying to classify ? 
    do : 0.5 Dropout probability
    l2_strenght : The L2 regularization strength
    
    output : The full pretrained_model that takes images of size (224, 224) and an additional vector
    of size vec_size as input
    """
    
    ### top_aux_model takes the vectorized text as input
    top_aux_model = Sequential()
    top_aux_model.add(Dense(vec_size, input_shape=(vec_size,), name='aux_input'))

    ### top_model takes output from VGG conv and then adds 2 hidden layers
    top_model = Sequential()
    top_model.add(Flatten(input_shape=pretrained_model.output_shape[1:], name='top_flatter'))
    top_model.add(Dense(256, activation='relu', name='top_relu', W_regularizer=l2(l2_strength)))
    top_model.add(Dropout(do))
    top_model.add(Dense(256, activation='sigmoid', name='top_sigmoid', W_regularizer=l2(l2_strength)))

    ### this is than added to the VGG conv-pretrained_model
    pretrained_model.add(top_model)
    
    ### here we merge 'pretrained_model' that creates features from images with 'top_aux_model'
    ### that are the bag of words features extracted from the text. 
    merged = Concatenate([pretrained_model, top_aux_model])
    print(merged.input_shape, merged.output_shape)
    ### final_model takes the combined feature vectors and add a sofmax classifier to it
    final_model = Sequential()
    for layer in merged.layers:
        final_model.add(layer)
    #final_model.add(merged)
    final_model.add(Dropout(do))
    final_model.add(Dense(1, activation='sigmoid'))

    return final_model

In [9]:
tmp = VGG16(weights='/Users/Account/Drink/kaggle/avito/pretrained_models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', 
                        include_top=False, 
                        input_shape=(224, 224, 3))
tmp.o
pretrained_model = Sequential()
for layer in tmp.layers:
    pretrained_model.add(layer)
del tmp
gc.collect()

7745

In [18]:
pretrained_model.output_shape

(None, 256)

In [17]:
model = make_final_model(pretrained_model, 2500)

ValueError: Input 0 is incompatible with layer top_flatter: expected min_ndim=3, found ndim=2

In [ ]:
model.compile(optimizer = 'adam',
              loss= binary_crossentropy,
              metrics=['accuracy'])

In [25]:
img_width = 224; img_height = 224
# build the VGG16 network
model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height, 3), name='image_input'))

model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
print(model.output_shape)

/Users/Account/.virtualenvs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="conv1_1")`
  
/Users/Account/.virtualenvs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="conv1_2")`
  
/Users/Account/.virtualenvs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", name="conv2_1")`
  if sys.path[0] == '':
/Users/Account/.virtualenvs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", name="conv2_2")`
  
/Users/Account/.virtualenvs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Co

(None, 7, 7, 512)


/Users/Account/.virtualenvs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu", name="conv5_1")`
/Users/Account/.virtualenvs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu", name="conv5_2")`
/Users/Account/.virtualenvs/kaggle/lib/python3.6/site-packages/ipykernel_launcher.py:38: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu", name="conv5_3")`
